In [38]:
import pandas as pd
import plotly.express as px

In [39]:
df_principal = pd.read_excel('acoes_pura.xlsx', sheet_name='Principal')

In [40]:
df_total_acoes = pd.read_excel('acoes_pura.xlsx', sheet_name='Total_de_acoes')

df_total_acoes = df_total_acoes.rename(columns={'Código':'ativo','Qtde. Teórica':'qtde_teorica'}).copy()

#df_total_acoes.head(10)

In [41]:
df_ticker = pd.read_excel('acoes_pura.xlsx', sheet_name='Ticker')

df_ticker = df_ticker.rename(columns={'Ticker':'ativo','Nome':'nome'}).copy()

#df_ticker.head(10)

In [42]:
df_chatgpt = pd.read_excel('acoes_pura.xlsx', sheet_name='ChatGPT')

df_chatgpt = df_chatgpt.rename(columns={'Nome da empresa':'nome','Segmento':'segmento','Idade (anos)':'idade'}).copy()

#df_chatgpt.head(10)

In [43]:
df_principal = df_principal[['Ativo', 'Data', 'Último (R$)', 'Var. Dia (%)']].copy()

#df_principal.head(10)

In [44]:
df_principal = df_principal.rename(columns={'Último (R$)':'valor_final','Var. Dia (%)':'var_dia_pct'}).copy()

#df_principal

In [45]:
df_principal = df_principal.rename(columns={'Ativo':'ativo','Data':'data'})

#df_principal

In [46]:
df_principal['var_pct'] = df_principal['var_dia_pct'] / 100

#df_principal

In [47]:
df_principal['valor_inicial'] = df_principal['valor_final'] / (1 + df_principal['var_pct'])

#df_principal

In [48]:
df_principal = df_principal.merge(df_total_acoes, left_on='ativo',right_on='ativo',how='left')

#df_principal

In [49]:
#df_principal = df_principal.drop(columns=['Código'])

#Remover coluna do DataFrame

In [50]:
df_principal = df_principal.drop_duplicates('ativo')

In [51]:
df_principal

,ativo,data,valor_final,var_dia_pct,var_pct,valor_inicial,qtde_teorica
0,USIM5,2024-01-26,9.50,5.20,0.05,9.03,515117391.00
1,CMIN3,2024-01-26,6.82,2.40,0.02,6.66,1110559345.00
2,PETR3,2024-01-26,41.96,2.19,0.02,41.06,2379877655.00
3,SUZB3,2024-01-26,52.91,2.04,0.02,51.85,683452836.00
4,CPFE3,2024-01-26,37.10,2.03,0.02,36.36,187732538.00
...,...,...,...,...,...,...,...
76,CRFB3,2024-01-26,10.71,-2.45,-0.02,10.98,533990587.00
77,BHIA3,2024-01-26,8.70,-2.46,-0.02,8.92,94843047.00
78,RENT3,2024-01-26,56.24,-3.63,-0.04,58.36,853202347.00
79,CVCB3,2024-01-26,3.07,-4.36,-0.04,3.21,525582771.00


In [52]:
df_principal['variacao_rs'] = (df_principal['valor_final']-df_principal['valor_inicial']) * df_principal['qtde_teorica']

In [53]:
pd.options.display.float_format = '{:.2f}'.format

In [54]:
df_principal['qtde_teorica'] = df_principal['qtde_teorica'].astype(int)

In [55]:
df_principal['resultado'] = df_principal['variacao_rs'].apply(lambda x: 'Subiu' if x > 0 else ('Desceu 'if x < 0 else 'Estável'))

In [56]:
df_principal = df_principal.rename(columns={'Resultado':'resultado'}).copy()

In [57]:
df_principal = df_principal.merge(df_ticker,left_on='ativo',right_on='ativo',how='left')

In [58]:
df_principal = df_principal.merge(df_chatgpt,left_on='nome',right_on='nome',how='left')

In [59]:
df_principal['resultado_idade'] = df_principal['idade'].apply(lambda x: 'Acima de 100' if x > 100 else ('Abaixo de 50' if x < 50 else 'Entre 50 e 100'))

In [60]:
#Análises

maior_variacao = df_principal['variacao_rs'].max()

menor_variacao = df_principal['variacao_rs'].min()

media_variacao = df_principal['variacao_rs'].mean()

media_variacao_subiu = df_principal[df_principal['resultado']=='Subiu']['variacao_rs'].mean()

media_variacao_desceu = df_principal[df_principal['resultado']=='Desceu ']['variacao_rs'].mean()

print(f'A Maior variação: R$ {maior_variacao:,.2f}')

print(f'A Menor variação: R$ {menor_variacao:,.2f}')

print(f'A Média de variação: R$ {media_variacao:,.2f}')

print(f'A Média de variação dos que Subiram: R$ {media_variacao_subiu:,.2f}')

print(f'A Média de variação dos que Desceram: R$ {media_variacao_desceu:,.2f}')


A Maior variação: R$ 4,762,926,995.25
A Menor variação: R$ -1,807,432,634.46
A Média de variação: R$ 218,008,898.33
A Média de variação dos que Subiram: R$ 538,720,638.36
A Média de variação dos que Desceram: R$ -186,265,310.46


In [61]:
df_principal_subiu = df_principal[df_principal['resultado']=='Subiu']

df_principal_subiu = df_principal_subiu.drop(columns='resultado')

In [62]:
#df_principal_subiu

In [63]:
df_principal_desceu = df_principal[df_principal['resultado']=='Desceu ']

df_principal_desceu = df_principal_desceu.drop(columns='resultado')

In [64]:
#df_principal_desceu

In [65]:
df_analise_segmento = df_principal_subiu.groupby('segmento')['variacao_rs'].sum().reset_index()

#df_analise_segmento

In [66]:
df_analise_saldo = df_principal.groupby('resultado')['variacao_rs'].sum().reset_index()

#df_analise_saldo

In [67]:
fig_analise_saldo = px.bar(df_analise_saldo, x='resultado' ,y='variacao_rs',text='variacao_rs', title='Variação Resultado por R$')

fig_analise_saldo.update_traces(texttemplate='%{text:,.2f}', textposition='inside')

fig_analise_saldo.update_layout(width=800, height=500)

fig_analise_saldo.show()

In [68]:
df_analise_idade = df_principal_subiu.groupby('resultado_idade')['variacao_rs'].sum().reset_index()

df_analise_idade

,resultado_idade,variacao_rs
0,Abaixo de 50,5485660198.58
1,Acima de 100,992503730.82
2,Entre 50 e 100,19380426711.95


In [69]:
fig_analise_idade = px.bar(df_analise_idade, x='resultado_idade',y='variacao_rs',text='variacao_rs',title='Variação Resultado por Idade')

fig_analise_idade.update_traces(texttemplate='%{text:,.2f}', textposition='inside')

fig_analise_idade.update_layout(width=800, height=500)

fig_analise_idade.show()

In [70]:
df_analise_segmento_subiu = df_principal[df_principal['resultado']=='Subiu'].groupby('segmento')['variacao_rs'].sum().reset_index()

#df_analise_segmento_subiu

In [71]:
fig_analise_segmento_subiu = px.pie(df_analise_segmento_subiu, names='segmento',values='variacao_rs',title='Análise de Segmento')

fig_analise_segmento_subiu.update_layout(width=600, height=500)

fig_analise_segmento_subiu.show()